In [ ]:
import pandas as pd

def describe_df(df):
    """
    Genera un DataFrame resumen con información descriptiva de cada variable.

    El DataFrame resultante contiene una columna por cada variable del DataFrame
    original y las siguientes filas: tipo de dato, porcentaje de valores nulos,
    número de valores únicos y porcentaje de cardinalidad.

    Argumentos:
    df (pandas.DataFrame): DataFrame de entrada.

    Retorna:
    pandas.DataFrame: DataFrame resumen con el formato especificado.
    """
      # Comprobación de que la entrada es un DataFrame
    if not isinstance(df, pd.DataFrame):
        print("Debes introducir un pandas DataFrame válido.")
        return None

    n_filas = len(df)   # Número de filas del DataFrame

 # Listas para almacenar la información de cada columna
    data_type = []
    missings = []
    unique_values = []
    cardin = []

    for col in df.columns:
        data_type.append(df[col].dtype)                         # Tipo de dato
        missings.append(round(df[col].isna().mean() * 100, 2))  # % de nulos
        unique = df[col].nunique()                              # Valores únicos
        unique_values.append(unique)
        cardin.append(round((unique / n_filas) * 100, 2))       # % cardinalidad

 # Construcción del DataFrame resumen
    resumen = pd.DataFrame(
        [
            data_type,
            missings,
            unique_values,
            cardin
        ],
        index=["DATA_TYPE", "MISSINGS (%)", "UNIQUE_VALUES", "CARDIN (%)"],
        columns=df.columns
    )

    return resumen


In [4]:
def get_features_num_regression(df, target_col, umbral_corr, pvalue=None):
    """
    Devuelve columnas numéricas con correlación alta respecto a un target para regresión.

    Selecciona las columnas numéricas del DataFrame cuya correlación (Pearson) con `target_col`
    sea superior en valor absoluto a `umbral_corr`. Si `pvalue` no es None, además exige que
    la correlación sea estadísticamente significativa (p-valor <= pvalue).

    Argumentos:
    df (pandas.DataFrame): DataFrame de entrada.
    target_col (str): Nombre de la columna objetivo (debe ser numérica y con alta cardinalidad).
    umbral_corr (float): Umbral de correlación absoluta, entre 0 y 1.
    pvalue (float | None): Umbral de p-valor (entre 0 y 1). Si es None, no se filtra por significancia.

    Retorna:
    list | None: Lista de columnas numéricas que cumplen el criterio o None si hay entradas inválidas.
    """


    # Check básico: df debe ser DataFrame
    if not isinstance(df, pd.DataFrame):
        print("df debe ser un pandas.DataFrame.")
        return None

    # Check básico: target_col debe existir
    if not isinstance(target_col, str) or target_col not in df.columns:
        print("target_col debe ser el nombre (str) de una columna existente del DataFrame.")
        return None

    # Check básico: umbral_corr entre 0 y 1
    if not isinstance(umbral_corr, (int, float)) or not (0 <= umbral_corr <= 1):
        print("umbral_corr debe ser un número entre 0 y 1.")
        return None

    # Check básico: pvalue None o entre 0 y 1
    if pvalue is not None and (not isinstance(pvalue, (int, float)) or not (0 <= pvalue <= 1)):
        print("pvalue debe ser None o un número entre 0 y 1.")
        return None

    # Check: el target debe ser numérico
    if not pd.api.types.is_numeric_dtype(df[target_col]):
        print("target_col debe referenciar una columna numérica.")
        return None

    # Check: el target debe tener alta cardinalidad (criterio simple)
    if df[target_col].nunique(dropna=True) <= 10:
        print("target_col no parece una variable de regresión (baja cardinalidad).")
        return None

    # Seleccionar columnas numéricas y quitar el target de candidatas
    num_cols = df.select_dtypes(include="number").columns.tolist()
    if target_col in num_cols:
        num_cols.remove(target_col)

    # Si no hay candidatas numéricas
    if len(num_cols) == 0:
        return []

    # Caso sin pvalue: filtrar solo por correlación
    if pvalue is None:
        corr = df[num_cols].corrwith(df[target_col]).dropna()
        return corr[corr.abs() > umbral_corr].index.tolist()

    # Caso con pvalue: filtrar por correlación y significancia
    try:
        from scipy.stats import pearsonr
    except Exception:
        print("No se pudo importar scipy para calcular p-valores. Usa pvalue=None o instala scipy.")
        return None

    selected = []
    for col in num_cols:
        tmp = df[[target_col, col]].dropna()

        # Pearson necesita variabilidad y suficientes datos
        if len(tmp) < 3:
            continue
        if tmp[target_col].nunique() < 2 or tmp[col].nunique() < 2:
            continue

        r, p = pearsonr(tmp[target_col], tmp[col])

        # Filtrar por umbral de correlación y p-valor
        if abs(r) > umbral_corr and p <= pvalue:
            selected.append(col)

    return selected
